In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel

In [3]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')
train_paths += glob('../feature/org_use/526*_train.gz')
test_paths  += glob('../feature/org_use/526*_test.gz')
train_paths += glob('../feature/raw_use/ker__uid*_train.gz')
test_paths  += glob('../feature/raw_use/ker__uid*_test.gz')

train_paths = [path for path in train_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('fill__M')
#                or path.count('V')
#                or path.count('C')
               or path.count('D1_')
#                or (path.count('526') and path.count('mean'))
#                or path.count('uid_')
               or path.count('uid2_t')
               or path.count('uid3_t')
               or path.count('uid4_t')
#                or path.count('uid5_t')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('domain')
#                or path.startswith('TransactionAmt')
               or path.count('Product')
#                or path.count('hour')
#                or path.count('zone')
#                or path.count('500')
               )
#                and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
               and not path.count('130')
#                or path.count('500')
              ]
test_paths = [path for path in test_paths 
               if (path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('fill__M')
#                or path.count('V')
#                or path.count('C')
               or path.count('D1_')
#                or (path.count('526') and path.count('mean'))
#                or path.count('uid_')
               or path.count('uid2_t')
               or path.count('uid3_t')
               or path.count('uid4_t')
#                or path.count('uid5_t')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('domain')
#                or path.startswith('TransactionAmt')
               or path.count('Product')
#                or path.count('hour')
#                or path.count('zone')
#                or path.count('500')
#                or path.count('500')
               )
#                and not path.count('fill')
               and not path.count('bin')
               and not path.count('129')
               and not path.count('130')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)
data = pd.concat([df_train, df_test], axis=0, ignore_index=True)
train_length = df_train.shape[0]

In [11]:
START_DATE = '2017-12-01'
# START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')

data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
data['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
data['date'] = data['datetime'].map(lambda x: x.date())
data['DT-M'] = data['date'].map(lambda x: str(x)[:7])

list_regist = []
for d, diff in tqdm(data[['date', 'D1']].values):
    if diff < 999999:
        regist = date_add_days(d, -1*diff)
    else:
        regist = date_add_days(d, 0)
    list_regist.append(str(regist))

data['Regist_date'] = list_regist
data['DT-M'] = data['date'].map(lambda x: str(x)[:7])

100%|██████████| 1097231/1097231 [00:07<00:00, 151129.56it/s]


In [18]:
cols_uid = [col for col in data.columns if col.count('uid')]
col_pd = 'ProductCD'

In [31]:
#========================================================================
# Future
#========================================================================

dir_save = 'valid_use'
list_dtm = sorted(data['DT-M'].value_counts().index.tolist())

for base_key in cols_uid:
    base_key = [base_key, 'Regist_date']
    
    base = data[base_key + ['DT-M']]
    
    list_df = []

    for this, future in zip(tqdm(list_dtm), list_dtm[1:]):
        df_this = data[data['DT-M']==this]
        df_future = data[data['DT-M']==future]
        
        df_this = df_this.groupby(base_key)['datetime'].agg({
            'this_count': 'count'
        })
        
        df_future = df_future.groupby(base_key)['datetime'].agg({
            'future_count': 'count'
        })
        
        tmp = pd.concat([df_this, df_future], axis=1)
        fname = f"{'-'.join(base_key)}__future_Transaction_count_ratio"
        tmp[fname] = tmp['future_count'].fillna(0) / (tmp['this_count'].fillna(0) + 1)
        tmp['DT-M'] = this

        list_df.append(tmp)
        
    df = pd.concat(list_df, axis=0)
    base = base.merge(df, on=base_key+['DT-M'], how='left')
    base[fname].fillna(0, inplace=True)
    
    train = base.iloc[:len(df_train)]
    test  = base.iloc[len(df_train):]
    
    save_feature(train[[fname]], '535', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
    save_feature(test[[fname]],  '535', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)




  0%|          | 0/13 [00:00<?, ?it/s]


  8%|▊         | 1/13 [00:01<00:14,  1.21s/it]


 15%|█▌        | 2/13 [00:02<00:12,  1.11s/it]


 23%|██▎       | 3/13 [00:03<00:10,  1.08s/it]


 31%|███       | 4/13 [00:04<00:09,  1.07s/it]


 38%|███▊      | 5/13 [00:05<00:08,  1.05s/it]


 46%|████▌     | 6/13 [00:05<00:06,  1.11it/s]


 54%|█████▍    | 7/13 [00:06<00:04,  1.24it/s]


 62%|██████▏   | 8/13 [00:07<00:03,  1.25it/s]


 69%|██████▉   | 9/13 [00:07<00:03,  1.25it/s]


 77%|███████▋  | 10/13 [00:08<00:02,  1.18it/s]


 85%|████████▍ | 11/13 [00:09<00:01,  1.18it/s]


 92%|█████████▏| 12/13 [00:10<00:00,  1.14it/s]

(590540,) | ker__uid2-Regist_date__future_Transaction_count_ratio
(506691,) | ker__uid2-Regist_date__future_Transaction_count_ratio






  0%|          | 0/13 [00:00<?, ?it/s]



  8%|▊         | 1/13 [00:01<00:17,  1.50s/it]



 15%|█▌        | 2/13 [00:02<00:15,  1.40s/it]



 23%|██▎       | 3/13 [00:03<00:13,  1.33s/it]



 31%|███       | 4/13 [00:05<00:11,  1.28s/it]



 38%|███▊      | 5/13 [00:06<00:10,  1.25s/it]



 46%|████▌     | 6/13 [00:06<00:07,  1.06s/it]



 54%|█████▍    | 7/13 [00:07<00:05,  1.04it/s]



 62%|██████▏   | 8/13 [00:08<00:04,  1.02it/s]



 69%|██████▉   | 9/13 [00:09<00:04,  1.03s/it]



 77%|███████▋  | 10/13 [00:10<00:03,  1.01s/it]



 85%|████████▍ | 11/13 [00:11<00:02,  1.06s/it]



 92%|█████████▏| 12/13 [00:13<00:01,  1.11s/it]

(590540,) | ker__uid3-Regist_date__future_Transaction_count_ratio
(506691,) | ker__uid3-Regist_date__future_Transaction_count_ratio







  0%|          | 0/13 [00:00<?, ?it/s]




  8%|▊         | 1/13 [00:01<00:21,  1.80s/it]




 15%|█▌        | 2/13 [00:03<00:18,  1.65s/it]




 23%|██▎       | 3/13 [00:04<00:15,  1.59s/it]




 31%|███       | 4/13 [00:06<00:14,  1.65s/it]




 38%|███▊      | 5/13 [00:07<00:12,  1.56s/it]




 46%|████▌     | 6/13 [00:08<00:09,  1.30s/it]




 54%|█████▍    | 7/13 [00:09<00:06,  1.11s/it]




 62%|██████▏   | 8/13 [00:10<00:05,  1.15s/it]




 69%|██████▉   | 9/13 [00:11<00:04,  1.15s/it]




 77%|███████▋  | 10/13 [00:12<00:03,  1.16s/it]




 85%|████████▍ | 11/13 [00:14<00:02,  1.25s/it]




 92%|█████████▏| 12/13 [00:15<00:01,  1.32s/it]

(590540,) | ker__uid4-Regist_date__future_Transaction_count_ratio
(506691,) | ker__uid4-Regist_date__future_Transaction_count_ratio


In [11]:
list_base_key = cols_uid
col_cat = 'ProductCD'
dir_save = 'valid_use'

for base_key in list_base_key:

    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
    
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    base = data[base_key + [col_cat]]
    
    all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
        f'count': 'count'
    })
    
    part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
        f'count_cat': 'count'
    })
    
    tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)
    
    tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
    tmp.drop(['count', 'count_cat'], axis=1, inplace=True)
    
#     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
#     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
#     result.columns = cols_save

    df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
    fname = f"cnt_ratio-{col_cat}-{name_base}"
    df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
    cols_save = [fname]
    
    base_train = df_feat.iloc[:df_train.shape[0]]
    base_test  = df_feat.iloc[df_train.shape[0]:]
    
    save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
    save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | cnt_ratio-ProductCD-ker__uid2-Regist_date
(506691,) | cnt_ratio-ProductCD-ker__uid2-Regist_date
(590540,) | cnt_ratio-ProductCD-ker__uid3-Regist_date
(506691,) | cnt_ratio-ProductCD-ker__uid3-Regist_date
(590540,) | cnt_ratio-ProductCD-ker__uid4-Regist_date
(506691,) | cnt_ratio-ProductCD-ker__uid4-Regist_date


In [6]:
# list_base_key = cols_uid
list_base_key = cols_card_addr
cat_list = cols_M + cols_domain + cols_501 + cols_time + cols_500
dir_save = 'valid_use'


for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    if str(type(base_key)).count('tuple'):
        base_key = list(base_key)
    else:
        base_key = [base_key]
        
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    for col_cat in tqdm(cat_list):

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)

        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
        cols_save = [fname]

        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(506691,) | cnt_ratio-500__device_country__screen_ratio-fill__cnt_card5-addr1-Regist_date


 41%|████      | 62/152 [03:56<07:25,  4.95s/it]

(590540,) | cnt_ratio-500__device_country__screen_type-fill__cnt_card5-addr1-Regist_date
(506691,) | cnt_ratio-500__device_country__screen_type-fill__cnt_card5-addr1-Regist_date


 41%|████▏     | 63/152 [04:01<07:18,  4.92s/it]

(590540,) | cnt_ratio-500__device_country__screen_width-fill__cnt_card5-addr1-Regist_date
(506691,) | cnt_ratio-500__device_country__screen_width-fill__cnt_card5-addr1-Regist_date


...

 99%|█████████▊| 150/152 [08:22<00:08,  4.33s/it]

(590540,) | cnt_ratio-500__screen_width-fill__cnt_card6-addr1-Regist_date
(506691,) | cnt_ratio-500__screen_width-fill__cnt_card6-addr1-Regist_date


 99%|█████████▉| 151/152 [08:26<00:04,  4.35s/it]

(590540,) | cnt_ratio-500__version_id_30-fill__cnt_card6-addr1-Regist_date
(506691,) | cnt_ratio-500__version_id_30-fill__cnt_card6-addr1-Regist_date


100%|██████████| 152/152 [08:32<00:00,  4.64s/it]


In [ ]:
def _calc_agg_category_func(df, key, value):
    new_fe_name_sample = f'{value}_groupby_{"_".join(key)}_n_sample'
    new_fe_name_ratio = f'{value}_groupby_{"_".join(key)}_ratio'
    tmp = df.groupby(key+[value]).size().rename(new_fe_name_sample).reset_index()
    tmp2 = tmp.groupby(key).size().rename("n_total").reset_index()
    result = pd.merge(tmp, tmp2, on=key, how="left")
    result[new_fe_name_ratio] = result["n_total"] / result[new_fe_name_sample]
    # concat
    df = pd.merge(df, result, on=key+[value], how="left")
    return df[[new_fe_name_sample, new_fe_name_ratio]]

In [ ]:
list_base_key = cols_uid
cat_list = cols_M
dir_save = 'valid_use'

for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
        
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    for col_cat in cat_list:

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)

        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
        cols_save = [fname]

        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

In [23]:
list_base_key = cols_uid
cat_list = cols_domain
dir_save = 'valid_use'

for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
        
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    for col_cat in cat_list:

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)

        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
        cols_save = [fname]

        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | cnt_ratio-P_emaildomain-ker__uid2-Regist_date
(506691,) | cnt_ratio-P_emaildomain-ker__uid2-Regist_date
(590540,) | cnt_ratio-R_emaildomain-ker__uid2-Regist_date
(506691,) | cnt_ratio-R_emaildomain-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__P_emaildomain_prefix-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__P_emaildomain_prefix-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__P_emaildomain_suffix-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__P_emaildomain_suffix-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__R_emaildomain_prefix-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__R_emaildomain_prefix-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__R_emaildomain_suffix-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__R_emaildomain_suffix-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__cnt_P_emaildomain-ker__uid2-Regist_date
(506691,) | cnt_ratio-fill__cnt_P_emaildomain-ker__uid2-Regist_date
(590540,) | cnt_ratio-fill__cnt_R_emaildomain-ker__uid2-Regist_d

In [24]:
list_base_key = cols_uid
cat_list = cols_time
dir_save = 'valid_use'

for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
        
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    for col_cat in cat_list:

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)

        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
        cols_save = [fname]

        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)

(590540,) | cnt_ratio-hour-ker__uid2-Regist_date
(506691,) | cnt_ratio-hour-ker__uid2-Regist_date
(590540,) | cnt_ratio-time_zone-ker__uid2-Regist_date
(506691,) | cnt_ratio-time_zone-ker__uid2-Regist_date
(590540,) | cnt_ratio-hour-ker__uid3-Regist_date
(506691,) | cnt_ratio-hour-ker__uid3-Regist_date
(590540,) | cnt_ratio-time_zone-ker__uid3-Regist_date
(506691,) | cnt_ratio-time_zone-ker__uid3-Regist_date
(590540,) | cnt_ratio-hour-ker__uid4-Regist_date
(506691,) | cnt_ratio-hour-ker__uid4-Regist_date
(590540,) | cnt_ratio-time_zone-ker__uid4-Regist_date
(506691,) | cnt_ratio-time_zone-ker__uid4-Regist_date


In [ ]:
list_base_key = cols_uid
cat_list = cols_501
dir_save = 'valid_use'

for base_key in list_base_key:
    
    if str(type(base_key)).count('list'):
        pass
    else:
        base_key = [base_key]
        
    base_key += ['Regist_date']
    name_base = '-'.join(base_key)
    
    for col_cat in cat_list:

        base = data[base_key + [col_cat]]

        all_cnt = data.groupby(base_key, as_index=False)[col_cat].agg({
            f'count': 'count'
        })

        part_cnt = data.groupby(base_key + [col_cat], as_index=False)[col_cat].agg({
            f'count_cat': 'count'
        })

        tmp = all_cnt.merge(part_cnt[base_key + [f"count_cat", col_cat]], how='inner', on=base_key)

        tmp['cat_ratio'] = tmp['count_cat'] / tmp['count']
        tmp.drop(['count', 'count_cat'], axis=1, inplace=True)

    #     result = pd.pivot_table(data=tmp, columns=col_cat, index=base_key)
    #     cols_save = [f"cnt_ratio-{col_cat}-{col[1]}-{name_base}" for col in result.columns]
    #     result.columns = cols_save

        df_feat = base.merge(tmp, how='left', on=base_key + [col_cat])
        fname = f"cnt_ratio-{col_cat}-{name_base}"
        df_feat.rename(columns={'cat_ratio': fname}, inplace=True)
        cols_save = [fname]

        base_train = df_feat.iloc[:df_train.shape[0]]
        base_test  = df_feat.iloc[df_train.shape[0]:]

        save_feature(base_train[cols_save], '534', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
        save_feature(base_test[cols_save],  '534', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)